In [44]:
##BECAUSE RF PU HAS NO PASS CALLS

In [1]:
import gzip
import os
from scipy.stats import binomtest

# --- Paths ---
input_path  = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/TruthSet/Collection/0.calls_shortread/PU_longread/"
output_path = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/1.Illumina_PU20/"

# --- Output files for removed variants ---
out_removed     = open("MT.ins_remove.txt", 'w')
out_del_removed = open("MT.del_remove.txt", 'w')

dic_step1  = {}
dic_altzero = {}
dic_VAF    = {}

modes = ["MT", "STK", "VN", "RF"]

# --- Collect AF and OtherAlt from INFO field ---
def collect_AF(file):
    altcount_dict = {}
    otheralt_dict = {}
    line_dict     = {}
    for line in file:
        if not line or line[0] == '#':
            continue
        fields = line.rstrip("\n").split('\t')
        if len(fields) < 8:
            continue
        chrom, pos, ref, alt, filt, info_field = fields[0], fields[1], fields[3], fields[4], fields[6], fields[7]
        if filt == "PASS" or filt == '.':
            variant = f"{chrom}:{pos}:{ref}:{alt}"
            line_dict[variant] = line.strip()
            for element in info_field.split(';'):
                if element.startswith('DTV=') and 'POPAF' not in element:
                    try:
                        altcount_dict[variant] = float(element.split('=')[1])
                    except:
                        pass
                elif element.startswith('OtherAlt='):
                    otheralt_dict[variant] = element.split('=')[1].split('_')
    return altcount_dict, otheralt_dict, line_dict

# --- Build allele counts from OtherAlt ---
def compute_ranks_from_otheralt(ref, alt, otheralt_list):
    allele_counts = {}
    for entry in (otheralt_list or []):
        if '-' not in entry:
            continue
        a, c = entry.split('-', 1)
        try:
            cnt = int(c)
        except:
            continue
        allele_counts[a] = allele_counts.get(a, 0) + cnt
    allele_counts.setdefault(ref, 0)
    allele_counts.setdefault(alt, 0)
    def rank_of(a):
        ca = allele_counts.get(a, 0)
        return 1 + sum(1 for v in allele_counts.values() if v > ca)
    return allele_counts, rank_of(alt), rank_of(ref)

# --- Load RF reference variants ---
rf_ref_variants = set()
rf_ref_path = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/TruthSet/output/RUFUS/RUFUS.COLO829.Ill.ao20.norm.ind.vcf"
if os.path.exists(rf_ref_path):
    with gzip.open(rf_ref_path, 'rt') if rf_ref_path.endswith('.gz') else open(rf_ref_path, 'r') as rf_ref_file:
        for line in rf_ref_file:
            if line.startswith("#"):
                continue
            f = line.strip().split('\t')
            rf_ref_variants.add(f"{f[0]}:{f[1]}:{f[3]}:{f[4]}")

# --- Main loop ---
for mode in modes:
    tool = mode
    chromosomes = [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"] if mode in ["MT", "STK"] else [None]

    total_count_all = alt_in_t_count_all = valid_count_all = alt_in_bl_count_all = alt_not_in_t_count_all = 0
    cnt_candi_ins = cnt_candi_del = cnt_valid_ins = cnt_valid_del = 0

    output_file_path = os.path.join(output_path, "Valid_tools_ind", f"{mode}.Valid.vcf")
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with open(output_file_path, 'w') as output_file:
        for chrom in chromosomes:
            if chrom not in dic_step1:
                dic_step1[chrom] = {}
            if chrom not in dic_altzero:
                dic_altzero[chrom] = {}

            tumor_prefix = mode
            if chrom:
                tumor_suffix   = f"{chrom}_annoAllAlt_COLO829T_Hifi.vcf.gz"
                tumorSR_suffix = f"{chrom}_annoAllAlt_COLO829T_Ill_200X.vcf.gz"
                bloodSR_suffix = f"{chrom}_annoAllAlt_COLO829BL_Ill_230X.vcf.gz"
            else:
                tumor_suffix   = f"{mode}_annoAllAlt_COLO829T_Hifi.vcf.gz"
                tumorSR_suffix = f"{mode}_annoAllAlt_COLO829T_Ill_200X.vcf.gz"
                bloodSR_suffix = f"{mode}_annoAllAlt_COLO829BL_Ill_230X.vcf.gz"

            tumor_path   = os.path.join(input_path, tumor_prefix, tumor_suffix)
            tumorSR_path = os.path.join(input_path, tumor_prefix, tumorSR_suffix)
            bloodSR_path = os.path.join(input_path, tumor_prefix, bloodSR_suffix)

            with gzip.open(tumor_path, "rt") as tumor_file, \
                 gzip.open(tumorSR_path, "rt") as tumorSR_file, \
                 gzip.open(bloodSR_path, "rt") as bloodSR_file:

                tumor_alt, tumor_otheralt, tumor_lines     = collect_AF(tumor_file)
                tumorSR_alt, tumorSR_otheralt, _           = collect_AF(tumorSR_file)
                bloodSR_alt, bloodSR_otheralt, _           = collect_AF(bloodSR_file)

                total_count = alt_in_t_count = valid_count = alt_in_bl_count = alt_not_in_t_count = 0

                for variant in tumor_alt:
                    if mode == "RF" and variant not in rf_ref_variants:
                        continue

                    total_count += 1
                    parts = variant.split(':')
                    chrom_v, ref, alt = parts[0], parts[2], parts[3]

                    if len(ref) >= 51 or len(alt) >= 51:
                        continue
                    if len(ref) < len(alt):
                        cnt_candi_ins += 1
                    elif len(ref) > len(alt):
                        cnt_candi_del += 1

                    allele_counts_pb, alt_rank, ref_rank = compute_ranks_from_otheralt(
                        ref, alt, tumor_otheralt.get(variant)
                    )
                    ref_count_pb = allele_counts_pb.get(ref, 0)
                    alt_count_pb = allele_counts_pb.get(alt, 0)
                    depth_pb     = sum(allele_counts_pb.values())
                    vaf_pb       = alt_count_pb / depth_pb if depth_pb > 0 else 0.0

                    allele_counts_ill, _, _ = compute_ranks_from_otheralt(
                        ref, alt, tumorSR_otheralt.get(variant)
                    )
                    alt_count_ill = allele_counts_ill.get(alt, 0)
                    depth_ill     = sum(allele_counts_ill.values())
                    vaf_ill       = alt_count_ill / depth_ill if depth_ill > 0 else 0.0

                    allele_counts_bl, _, _ = compute_ranks_from_otheralt(
                        ref, alt, bloodSR_otheralt.get(variant)
                    )
                    alt_count_bl = allele_counts_bl.get(alt, 0)
                    depth_bl     = sum(allele_counts_bl.values())
                    vaf_bl       = alt_count_bl / depth_bl if depth_bl > 0 else 0.0

                    dic_VAF[variant] = {
                        "VAF_pb": vaf_pb,
                        "VAF_ill": vaf_ill,
                        "VAF_bl": vaf_bl
                    }

                    tumor_balance_ok = False
                    if depth_pb > 0:
                        p_binom = binomtest(alt_count_pb, depth_pb, p=0.5, alternative='two-sided').pvalue
                        if vaf_pb > 0.5 or p_binom > 0.05:
                            tumor_balance_ok = True

                    if (vaf_pb > 0) and (alt_rank == 1 or (ref_rank == 1 and alt_rank == 2)) \
                       and vaf_pb > 0.2 and vaf_ill > 0.2:
                        alt_in_t_count += 1
                        if vaf_bl < 0.05:
                            valid_count += 1
                            dic_step1[chrom_v].setdefault(variant, []).append(tool)
                            output_file.write(f"{tumor_lines[variant]}\n")
                            if len(ref) < len(alt):
                                cnt_valid_ins += 1
                            elif len(ref) > len(alt):
                                cnt_valid_del += 1
                        else:
                            alt_in_bl_count += 1
                            if len(ref) < len(alt):
                                out_removed.write(variant + "\n")
                            elif len(ref) > len(alt):
                                out_del_removed.write(variant + "\n")
                    else:
                        alt_not_in_t_count += 1
                        dic_altzero[chrom_v].setdefault(variant, []).append(tool)

                total_count_all     += total_count
                alt_in_t_count_all  += alt_in_t_count
                valid_count_all     += valid_count
                alt_in_bl_count_all += alt_in_bl_count
                alt_not_in_t_count_all += alt_not_in_t_count

    # --- Summary Output ---
    print(f"{mode} (Combined Chromosomes)")
    print(f"Total: {total_count_all}")
    print(f"Alt in T: {alt_in_t_count_all}")
    print(f"Alt not in T: {alt_not_in_t_count_all}")
    print(f"Valid: {valid_count_all}")
    print(f"Alt in BL (Illumina): {alt_in_bl_count_all}")
    print(f"Total, valid insertion: {cnt_candi_ins}, {cnt_valid_ins}")
    print(f"Total, valid deletion: {cnt_candi_del}, {cnt_valid_del}\n")

    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, "stats.txt"), 'a') as stats_file:
        stats_file.write(
            f"{tool}\tind\tCombined\t{total_count_all}\t{alt_in_t_count_all}\t"
            f"{valid_count_all}\t{alt_in_bl_count_all}\t{alt_not_in_t_count_all}\n"
        )
    print(f"{mode} done.")


MT (Combined Chromosomes)
Total: 2416
Alt in T: 1710
Alt not in T: 650
Valid: 1675
Alt in BL (Illumina): 35
Total, valid insertion: 1091, 770
Total, valid deletion: 1269, 905

MT done.
STK (Combined Chromosomes)
Total: 2095
Alt in T: 1328
Alt not in T: 767
Valid: 1194
Alt in BL (Illumina): 134
Total, valid insertion: 1009, 599
Total, valid deletion: 1086, 595

STK done.
VN (Combined Chromosomes)
Total: 800
Alt in T: 735
Alt not in T: 65
Valid: 729
Alt in BL (Illumina): 6
Total, valid insertion: 360, 334
Total, valid deletion: 440, 395

VN done.
RF (Combined Chromosomes)
Total: 2099
Alt in T: 1363
Alt not in T: 714
Valid: 1008
Alt in BL (Illumina): 355
Total, valid insertion: 816, 379
Total, valid deletion: 1261, 629

RF done.


In [45]:
import gzip
import os
from scipy.stats import binomtest

# --- Paths ---
input_path  = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/TruthSet/Collection/0.calls_shortread/PU_longread/"
output_path = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/1.Illumina_PU20/"

# --- Output files for removed variants ---
out_removed     = open("MT.ins_remove.txt", 'w')      # insertions failing BL filter
out_del_removed = open("MT.del_remove.txt", 'w')      # deletions failing BL filter

dic_step1  = {}
dic_altzero = {}
dic_VAF    = {}   # keep VAFs for later annotation

# --- Modes to process ---
modes = ["MT", "STK", "VN", "RF"]

# --- Collect AF and OtherAlt from INFO field ---
def collect_AF(file):
    altcount_dict = {}
    otheralt_dict = {}
    line_dict     = {}

    for line in file:
        if not line or line[0] == '#':
            continue
        fields = line.rstrip("\n").split('\t')
        if len(fields) < 8:
            continue
        chrom, pos, ref, alt, filt, info_field = fields[0], fields[1], fields[3], fields[4], fields[6], fields[7]
        if filt == "PASS" or filt == '.':  # Use only PASS calls
            variant = f"{chrom}:{pos}:{ref}:{alt}"
            line_dict[variant] = line.strip()
            for element in info_field.split(';'):
                if element.startswith('DTV=') and 'POPAF' not in element:
                    try:
                        altcount_dict[variant] = float(element.split('=')[1])
                    except:
                        pass
                elif element.startswith('OtherAlt='):
                    otheralt_dict[variant] = element.split('=')[1].split('_')
    return altcount_dict, otheralt_dict, line_dict

# --- Build allele counts from OtherAlt ---
def compute_ranks_from_otheralt(ref, alt, otheralt_list):
    allele_counts = {}
    for entry in (otheralt_list or []):
        if '-' not in entry:
            continue
        a, c = entry.split('-', 1)
        try:
            cnt = int(c)
        except:
            continue
        allele_counts[a] = allele_counts.get(a, 0) + cnt
    allele_counts.setdefault(ref, 0)
    allele_counts.setdefault(alt, 0)

    def rank_of(a):
        ca = allele_counts.get(a, 0)
        return 1 + sum(1 for v in allele_counts.values() if v > ca)

    return allele_counts, rank_of(alt), rank_of(ref)

# --- Main loop ---
for mode in modes:
    tool = mode

    if mode in ["MT", "STK"]:
        chromosomes = [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"]
    else:
        chromosomes = [None]

    total_count_all = alt_in_t_count_all = valid_count_all = alt_in_bl_count_all = alt_not_in_t_count_all = 0
    cnt_candi_ins = cnt_candi_del = cnt_valid_ins = cnt_valid_del = 0

    output_file_path = os.path.join(output_path, "Valid_tools_ind", f"{mode}.Valid.vcf")
    os.makedirs(os.path.dirname(output_file_path), exist_ok=True)

    with open(output_file_path, 'w') as output_file:
        for chrom in chromosomes:
            if chrom not in dic_step1:
                dic_step1[chrom] = {}
            if chrom not in dic_altzero:
                dic_altzero[chrom] = {}

            tumor_prefix = mode
            if chrom:
                tumor_suffix   = f"{chrom}_annoAllAlt_COLO829T_Hifi.vcf.gz"
                tumorSR_suffix = f"{chrom}_annoAllAlt_COLO829T_Ill_200X.vcf.gz"
                bloodSR_suffix = f"{chrom}_annoAllAlt_COLO829BL_Ill_230X.vcf.gz"
            else:
                tumor_suffix   = f"{mode}_annoAllAlt_COLO829T_Hifi.vcf.gz"
                tumorSR_suffix = f"{mode}_annoAllAlt_COLO829T_Ill_200X.vcf.gz"
                bloodSR_suffix = f"{mode}_annoAllAlt_COLO829BL_Ill_230X.vcf.gz"

            tumor_path   = os.path.join(input_path, tumor_prefix, tumor_suffix)
            tumorSR_path = os.path.join(input_path, tumor_prefix, tumorSR_suffix)
            bloodSR_path = os.path.join(input_path, tumor_prefix, bloodSR_suffix)

            with gzip.open(tumor_path, "rt") as tumor_file, \
                 gzip.open(tumorSR_path, "rt") as tumorSR_file, \
                 gzip.open(bloodSR_path, "rt") as bloodSR_file:

                tumor_alt, tumor_otheralt, tumor_lines     = collect_AF(tumor_file)
                tumorSR_alt, tumorSR_otheralt, _           = collect_AF(tumorSR_file)
                bloodSR_alt, bloodSR_otheralt, _           = collect_AF(bloodSR_file)

                total_count = alt_in_t_count = valid_count = alt_in_bl_count = alt_not_in_t_count = 0

                for variant in tumor_alt:
                    total_count += 1
                    parts = variant.split(':')
                    chrom_v, ref, alt = parts[0], parts[2], parts[3]

                    if len(ref) >= 51 or len(alt) >= 51:
                        continue

                    if len(ref) < len(alt):
                        cnt_candi_ins += 1
                    elif len(ref) > len(alt):
                        cnt_candi_del += 1

                    # --- PacBio tumor ---
                    allele_counts_pb, alt_rank, ref_rank = compute_ranks_from_otheralt(
                        ref, alt, tumor_otheralt.get(variant)
                    )
                    ref_count_pb = allele_counts_pb.get(ref, 0)
                    alt_count_pb = allele_counts_pb.get(alt, 0)
                    depth_pb     = sum(allele_counts_pb.values())
                    vaf_pb       = alt_count_pb / depth_pb if depth_pb > 0 else 0.0

                    # --- Illumina tumor ---
                    allele_counts_ill, _, _ = compute_ranks_from_otheralt(
                        ref, alt, tumorSR_otheralt.get(variant)
                    )
                    alt_count_ill = allele_counts_ill.get(alt, 0)
                    depth_ill     = sum(allele_counts_ill.values())
                    vaf_ill       = alt_count_ill / depth_ill if depth_ill > 0 else 0.0

                    # --- Illumina blood ---
                    allele_counts_bl, _, _ = compute_ranks_from_otheralt(
                        ref, alt, bloodSR_otheralt.get(variant)
                    )
                    alt_count_bl = allele_counts_bl.get(alt, 0)
                    depth_bl     = sum(allele_counts_bl.values())
                    vaf_bl       = alt_count_bl / depth_bl if depth_bl > 0 else 0.0

                    # --- Store VAFs ---
                    dic_VAF[variant] = {
                        "VAF_pb": vaf_pb,
                        "VAF_ill": vaf_ill,
                        "VAF_bl": vaf_bl
                    }

                    # --- Tumor balance check ---
                    tumor_balance_ok = False
                    if depth_pb > 0:
                        p_binom = binomtest(alt_count_pb, depth_pb, p=0.5, alternative='two-sided').pvalue
                        if vaf_pb > 0.5 or p_binom > 0.05:
                            tumor_balance_ok = True

                    # --- Filtering ---
                    if (vaf_pb > 0) and (alt_rank == 1 or (ref_rank == 1 and alt_rank == 2)) \
                       and vaf_pb > 0.2 and vaf_ill > 0.2:
                        alt_in_t_count += 1
                        if vaf_bl < 0.05:
                            valid_count += 1
                            dic_step1[chrom_v].setdefault(variant, []).append(tool)
                            output_file.write(f"{tumor_lines[variant]}\n")
                            if len(ref) < len(alt):
                                cnt_valid_ins += 1
                            elif len(ref) > len(alt):
                                cnt_valid_del += 1
                        else:
                            alt_in_bl_count += 1
                            if len(ref) < len(alt):
                                out_removed.write(variant + "\n")
                            elif len(ref) > len(alt):
                                out_del_removed.write(variant + "\n")
                    else:
                        alt_not_in_t_count += 1
                        dic_altzero[chrom_v].setdefault(variant, []).append(tool)

                total_count_all     += total_count
                alt_in_t_count_all  += alt_in_t_count
                valid_count_all     += valid_count
                alt_in_bl_count_all += alt_in_bl_count
                alt_not_in_t_count_all += alt_not_in_t_count

    # --- Print summary ---
    print(f"{mode} (Combined Chromosomes)")
    print(f"Total: {total_count_all}")
    print(f"Alt in T: {alt_in_t_count_all}")
    print(f"Alt not in T: {alt_not_in_t_count_all}")
    print(f"Valid: {valid_count_all}")
    print(f"Alt in BL (Illumina): {alt_in_bl_count_all}")
    print(f"Total, valid insertion: {cnt_candi_ins}, {cnt_valid_ins}")
    print(f"Total, valid deletion: {cnt_candi_del}, {cnt_valid_del}\n")

    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, "stats.txt"), 'a') as stats_file:
        stats_file.write(
            f"{tool}\tind\tCombined\t{total_count_all}\t{alt_in_t_count_all}\t"
            f"{valid_count_all}\t{alt_in_bl_count_all}\t{alt_not_in_t_count_all}\n"
        )
    print(f"{mode} done.")


KeyboardInterrupt: 

In [4]:

vcf_header = "##fileformat=VCFv4.1\n##INFO=<ID=SP,Number=1,Type=String,Description='Variant caller supported this variant'>\n#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\tCOLO829T\n"
##INFO=<ID=END,Number=1,Type=Integer,Description="End position (for use with symbolic alleles)">
#print (len(dic_raw_all))
l_chrom= [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"]

cnt = 0

out_snv = open("/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/1.Illumina_PU20/Step1_Validated_union_ind.vcf", 'w')
out_snv_invalid = open("/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/1.Illumina_PU20/Step1_NOTValidated_union_snv.ind", 'w')


out_snv.write(vcf_header)

for chr_each in l_chrom:
    
    dic_bypos = {} #For sorting pos
    l_bypos = []
    for info in dic_step1[chr_each]:
        s = info.split(':')
        chrom, pos, ref, alt = s[0], int(s[1]), s[2], s[3]
        dic_bypos[pos] = info
        l_bypos.append(pos)
    l_bypos.sort()
    
    l_pos_per_chr = []
    for pos in l_bypos:
        if pos not in l_pos_per_chr:
            cnt +=1
            l_pos_per_chr.append(pos)
            info = dic_bypos[pos]
            chrom, pos, ref, alt = s[0], s[1], s[2], s[3]
            s = info.split(':')
            chrom, pos, ref, alt = s[0], int(s[1]), s[2], s[3]

            l_tool= dic_step1[chrom][info]
            tool_support = ','.join(l_tool)
            out_snv.write("%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\t%s\n" %(chrom, str(pos), '.', ref, alt, '.', '.', 'SP='+ tool_support, '.\t.'))
            
print (cnt)

2059


In [36]:
print (dic_VAF)

{'chr1:2045585:C:CT': {'VAF_pb': 0.9666666666666667, 'VAF_ill': 4.933333333333334, 'VAF_bl': 0.0}, 'chr1:2759232:CT:C': {'VAF_pb': 0.0, 'VAF_ill': 0.06451612903225806, 'VAF_bl': 0.0967741935483871}, 'chr1:9364848:G:GTTGTTGTTGTTGGTTTTTC': {'VAF_pb': 0.9518072289156626, 'VAF_ill': 1.8674698795180722, 'VAF_bl': 0.0}, 'chr1:18560675:GAGA:G': {'VAF_pb': 0.02830188679245283, 'VAF_ill': 0.08490566037735849, 'VAF_bl': 0.0}, 'chr1:23778337:C:CCT': {'VAF_pb': 0.5056179775280899, 'VAF_ill': 2.033707865168539, 'VAF_bl': 0.25842696629213485}, 'chr1:24345240:GCCCCCTCCACACCTGTGC:G': {'VAF_pb': 0.0, 'VAF_ill': 0.012987012987012988, 'VAF_bl': 0.012987012987012988}, 'chr1:26657658:G:GA': {'VAF_pb': 0.52, 'VAF_ill': 1.76, 'VAF_bl': 0.02}, 'chr1:27196792:CG:C': {'VAF_pb': 0.9354838709677419, 'VAF_ill': 2.7204301075268815, 'VAF_bl': 0.0}, 'chr1:30375697:TC:T': {'VAF_pb': 0.9861111111111112, 'VAF_ill': 0.5555555555555556, 'VAF_bl': 0.0}, 'chr1:30835532:TATA:T': {'VAF_pb': 0.0, 'VAF_ill': 0.0625, 'VAF_bl': 0

In [6]:
import pysam

# File paths
input_vcf_path  = "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/2.PacBio_Valid/Step1_Validated_union_ind.vcf"
output_vcf_path = input_vcf_path[:-4] + "_VAF_RGN.vcf"

# --- Prepare headers ---
vcf_in = pysam.VariantFile(input_vcf_path, "r")
new_header = vcf_in.header.copy()

# Add contigs (chr1..22, X, Y) if missing
for c in [f"chr{i}" for i in range(1, 23)] + ["chrX", "chrY"]:
    if c not in new_header.contigs:
        new_header.contigs.add(c)

# Add INFO fields for VAFs and Region
if "VAF_Ill" not in new_header.info:
    new_header.info.add("VAF_Ill", 1, "Float", "Variant allele frequency in 200X Illumina COLO829T")
if "VAF_PB" not in new_header.info:
    new_header.info.add("VAF_PB", 1, "Float", "Variant allele frequency in 180X PacBio COLO829T")
if "VAF_BL" not in new_header.info:
    new_header.info.add("VAF_BL", 1, "Float", "Variant allele frequency in 230X Illumina COLO829BL")
if "RGN" not in new_header.info:
    new_header.info.add("RGN", 1, "String", "SMaHT region stratification: Easy, Difficult, or Extreme")

# --- Load region map ---
region_vcfs = {
    "Easy": "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/2.PacBio_Valid/Region_indel2/Step1_Validated_union_ind.SMaHT_easy_v2.bed.gz.vcf",
    "Difficult": "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/2.PacBio_Valid/Region_indel2/Step1_Validated_union_ind.SMaHT_difficult_v2.bed.gz.vcf",
    "Extreme": "/n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/2.PacBio_Valid/Region_indel2/Step1_Validated_union_ind.SMaHT_extreme_v2.bed.gz.vcf",
}

region_map = {}
for region, path in region_vcfs.items():
    with pysam.VariantFile(path) as rv:
        for rec in rv.fetch():
            if not rec.alts:
                continue
            variant_key = f"{rec.chrom}:{rec.pos}:{rec.ref}:{rec.alts[0]}"
            region_map[variant_key] = region

# --- Open output with the augmented header ---
vcf_out = pysam.VariantFile(output_vcf_path, "w", header=new_header)

# Helper to copy INFO with header awareness
def copy_info_with_header_awareness(src_rec, dst_rec, header):
    for key, val in src_rec.info.items():
        # skip undefined keys in header (defensive)
        if key not in header.info:
            continue
        expected_num = header.info[key].number  # can be int, 0/1, or 'A','R','.'
        try:
            # iterable values (tuple/list)
            if isinstance(val, (tuple, list)):
                if expected_num in ('A', 'R', '.'):
                    # header allows variable-length or per-allele arrays -> pass as-is
                    dst_rec.info[key] = list(val)  # list/tuple are both accepted
                elif isinstance(expected_num, int) and expected_num > 1:
                    # header expects fixed-length >1
                    if len(val) == expected_num:
                        dst_rec.info[key] = list(val)
                    else:
                        # length mismatch -> skip (safer than forcing)
                        continue
                elif isinstance(expected_num, int) and expected_num in (0, 1):
                    # header expects scalar or flag; only accept singletons
                    if len(val) == 1:
                        dst_rec.info[key] = val[0]
                    else:
                        # multi-value for scalar field -> skip
                        continue
                else:
                    # any other unexpected case -> skip
                    continue
            else:
                # scalar
                if expected_num in ('A', 'R', '.') or (isinstance(expected_num, int) and expected_num > 1):
                    # header expects an array but we have a scalar; wrap as single-element array
                    dst_rec.info[key] = [val]
                else:
                    dst_rec.info[key] = val
        except Exception:
            # last-resort: skip problematic keys rather than failing the whole run
            continue

# --- Process variants ---
for rec in vcf_in.fetch():
    if not rec.alts:
        continue
    chrom = rec.chrom
    pos   = str(rec.pos)
    ref   = rec.ref
    alt   = rec.alts[0]

    # Create a new record using the new header
    new_rec = vcf_out.new_record(
        contig=chrom,
        start=rec.start,
        stop=rec.stop,
        id=rec.id,
        alleles=rec.alleles,
        qual=rec.qual,
        filter=list(rec.filter.keys()) if rec.filter is not None else None
    )

    # Copy FORMAT and samples (keep COLO829T column)
    # If you also want all genotypes/fields copied:
    for sample in rec.samples:
        new_rec.samples[sample].update(rec.samples[sample])

    # Copy INFO fields safely
    copy_info_with_header_awareness(rec, new_rec, new_header)

    # Annotate with VAFs from dic_VAF (ensure dic_VAF is in scope from your first script)
    variant_key = ":".join([chrom, pos, ref, alt])
    if variant_key in dic_VAF:
        v = dic_VAF[variant_key]
        # NOTE: Your dic_VAF example shows counts (e.g., 296.0), not fractions.
        # If those are counts, either convert to VAF before writing, or rename fields to CNT_*.
        new_rec.info["VAF_Ill"] = float(v.get("VAF_ill", 0.0))
        new_rec.info["VAF_PB"]  = float(v.get("VAF_pb", 0.0))
        if "VAF_bl" in v:
            new_rec.info["VAF_BL"] = float(v.get("VAF_bl", 0.0))

    # Annotate region
    if variant_key in region_map:
        new_rec.info["RGN"] = region_map[variant_key]

    vcf_out.write(new_rec)

vcf_in.close()
vcf_out.close()
print(f"Output written to {output_vcf_path}")


Output written to /n/data1/hms/dbmi/park/jiny/SMaHT/COLO829/0.truthset/2.PacBio_Valid/Step1_Validated_union_ind_VAF_RGN.vcf


[W::vcf_parse] Contig 'chr1' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr2' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr3' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr4' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr5' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr6' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr7' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr8' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr9' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'chr10' is not defined i